In [3]:
import os 
import sys
import numpy as np
import argparse
from scipy.spatial.transform import Rotation as R 
import open3d as o3d 
# add catkin_ws context 
sys.path.append("/home/junting/catkin_ws/devel/lib/python3.9/site-packages")
sys.path.append("/home/junting/franka_ws/devel/lib/python3.9/site-packages")

from src.lmp import *
from src.env.true_grounding_env import TrueGroundingEnv
from src.configs.config import load_config
cfg_tabletop = load_config("perception_few_shot_gpt_3.5.yaml")
import rospy 
import rospkg
import jupyros as jr

from std_msgs.msg import String, Header
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion


In [4]:
########################################
# initialize environment
########################################
rospy.init_node('eval_code', log_level=rospy.DEBUG)
# get package root path 
pkg_root = rospkg.RosPack().get_path('instruct_to_policy')

# setup environment
env = TrueGroundingEnv(cfg_tabletop)
# env.reset()


## Ground truth perception and 3D fusion 

In [ ]:
# 3D fusion 
from src.perception.scene_manager import SceneManager
sensor_data = env.get_sensor_data()
sensor_data['detections_list'] = [{},{},{}]

scene_manager = SceneManager()
scene_manager.update_fusion(sensor_data)

In [ ]:
# get cropped point cloud
# drawer_bbox = env.get_3d_bbox('cabinet.drawer0')
# drawer_pcd = scene_manager.scene_tsdf_full.crop_cloud(
#     crop_center=(drawer_bbox[:3] + drawer_bbox[3:]) / 2,
#     crop_size=(drawer_bbox[3:] - drawer_bbox[:3])
# )
# cabinet_bbox = env.get_3d_bbox('cabinet')
# cabinet_pcd = scene_manager.scene_tsdf_full.crop_cloud(
#     crop_center=(cabinet_bbox[:3] + cabinet_bbox[3:]) / 2,
#     crop_size=(cabinet_bbox[3:] - cabinet_bbox[:3])
# )

In [ ]:
import open3d as o3d 
# o3d.io.write_point_cloud(os.path.join(pkg_root, 'data', 'drawer.pcd'), drawer_pcd)
# o3d.io.write_point_cloud(os.path.join(pkg_root, 'data', 'cabinet.pcd'), cabinet_pcd)

## Pick and Place

In [ ]:
env.reset()

In [5]:
pose_msg = env.parse_adaptive_shape_grasp_pose("apple")

In [ ]:
rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)

In [ ]:
env.open_gripper()

In [ ]:
env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)

In [ ]:
env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)

In [ ]:
env.close_gripper(width=0.05, force=30)

In [ ]:
env.attach_object("apple")

In [ ]:
place_pose = env.parse_place_pose(object_name="apple", receptacle_name="white_ceramic_plate")

In [ ]:
env.move_to_pose(place_pose)
env.open_gripper()

In [ ]:
env.detach_object("apple")

## Open Drawer

In [ ]:
jr.publish('/rviz/moveit/move_marker/goal_panda_hand_tcp', PoseStamped)

In [ ]:
env.reset()
# env.open_gripper()

In [ ]:
# env.get_obj_name_list()
# [bbox.object_id for bbox in env.gazebo_gt_bboxes]
env.get_3d_bbox("cabinet.handle_0")

In [ ]:
grasp_pose = env.parse_adaptive_shape_grasp_pose(object_name="cabinet.handle_0")
print(grasp_pose)

In [ ]:
env.publish_goal_to_marker(grasp_pose)

In [ ]:
env.grasp(grasp_pose)
# env.move_to_pose(grasp_pose)

In [ ]:
env.close_gripper()

In [ ]:
# generate a horizontal trajectory to open the drawer
grasp_position = np.array([grasp_pose.position.x, grasp_pose.position.y, grasp_pose.position.z])
pull_position = grasp_position + np.array([0.2, 0, 0]).astype(float)
pull_pose = Pose(position=Point(*pull_position), orientation=grasp_pose.orientation)

In [ ]:
env.move_to_pose(pull_pose)

In [ ]:
env.open_gripper()

In [ ]:
# test different grasp depth 
pose_msg = env.parse_adaptive_shape_grasp_pose(object_name="apple")

rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)
 
env.open_gripper()
 
# env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)
# env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)
env.close_gripper(width=0.05, force=30)
env.attach_object("apple")


In [ ]:
env.reset()

## Grasp preference

In [ ]:
from graspnetAPI import GraspGroup, Grasp
from grasp_detection.msg import Grasp as GraspMsg
from src.grasp_detection.utils import select_grasp_by_preference

# reload src.grasp_detection.utils 
import importlib
importlib.reload(sys.modules['src.grasp_detection.utils'])
from src.grasp_detection.utils import select_grasp_by_preference

In [ ]:
object_name = "white_and_pink_box"


In [ ]:
# ground truth preference of object 

# preferred_position is the center of the object
preferred_position = env.get_object_center_position(object_name)

# preferred_orientation is table surface 
preferred_plane_normal = np.array([0, 0, 1])


In [ ]:
# Manually call grasp detection model to get list of grasps

object_bbox = env.get_3d_bbox(object_name)    
sensor_data = env.get_sensor_data()

object_pcd = scene_manager.scene_tsdf_full.crop_cloud(
    crop_center=(object_bbox[:3] + object_bbox[3:]) / 2,
    crop_size=(object_bbox[3:] - object_bbox[:3])
)

bbox_center = (object_bbox[:3] + object_bbox[3:]) / 2
bbox_size = object_bbox[3:] - object_bbox[:3]
        
data = {
    # 'detections_list': detections_list,
    'bboxes_3d_dict':{
        object_name:{'center': bbox_center, 'size': bbox_size}
    }
}
data.update(sensor_data)

# call grasp detection service
grasp_candidates: List[GraspMsg] = env.grasp_model.predict(data)

In [ ]:
# Manually select a grasp from the list of grasps
grasp_idx, weighted_score = select_grasp_by_preference(grasp_candidates, preferred_position=preferred_position, preferred_plane_normal=preferred_plane_normal)

In [ ]:
# visualize all grasp candidates with object point cloud 
grasp_o3d_meshes = []

# rotate grasp orientation from gazebo gripper frame to anygrasp gripper frame
rot_anygrasp2gazebo = np.array([[0,0,1],[0,1,0],[-1,0,0]])
rot_gazebo2anygrasp = np.linalg.inv(rot_anygrasp2gazebo)

for i, grasp_msg in enumerate(grasp_candidates):
    quat = np.array([grasp_msg.grasp_pose.orientation.x, grasp_msg.grasp_pose.orientation.y, grasp_msg.grasp_pose.orientation.z, grasp_msg.grasp_pose.orientation.w])
    rotation_matrix = R.from_quat(quat).as_matrix()
    rotation_matrix = rotation_matrix @ rot_gazebo2anygrasp
    translation = np.array([grasp_msg.grasp_pose.position.x, grasp_msg.grasp_pose.position.y, grasp_msg.grasp_pose.position.z])
    # [score, width, height, depth, rotation_matrix, translation, object_id]
    grasp = Grasp(
        *[grasp_msg.grasp_score, grasp_msg.grasp_width, 0.02, grasp_msg.grasp_depth, rotation_matrix, translation, i]
    )
    grasp_o3d_meshes.append(grasp.to_open3d_geometry())
    
# change the selected grasp color to red 
grasp_o3d_meshes[grasp_idx].paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries([object_pcd, *grasp_o3d_meshes])    

In [ ]:
# predict the grasp pose of white_and_pink_box with anygrasp
pose_msg = env.parse_adaptive_shape_grasp_pose(object_name=object_name, preferred_position=preferred_position, preferred_plane_normal=preferred_plane_normal)
env.publish_goal_to_marker(pose_msg)

## Joint Prediction 

In [ ]:
env.reset()

In [ ]:
env.get_gt_bbox('cabinet.handle_0')

In [ ]:
# get handle 0 position 
handle_0_position = env.get_object_center_position("cabinet.handle_0")
print(handle_0_position)

In [ ]:
joint = env.get_object_joint_info(obj_name="cabinet", position=handle_0_position, type="any")
joint_axis = joint['joint_axis']

In [ ]:
env.open_gripper(width=0.10)

In [ ]:
# preferred_position: Optional(np.ndarray), prefered gripper tip point position 
# preferred_approach_direction: Optional(np.ndarray), prefered gripper approach direction
# preferred_plane_normal: Optional(np.ndarray), prefered gripper plane normal direction  

grasp_pose = env.parse_adaptive_shape_grasp_pose(object_name="cabinet.handle_0", preferred_position=handle_0_position, preferred_approach_direction=joint_axis)
env.publish_goal_to_marker(grasp_pose)

In [ ]:
env.grasp(grasp_pose, pre_grasp_approach=0.05, tentative_depth_list=[0])

In [ ]:
env.close_gripper()

In [ ]:
# generate a horizontal trajectory to open the drawer
def move_in_direction(axis: np.array, distance: float):
    current_pose = env.get_gripper_pose()
    target_pose = Pose()
    normalized_axis = np.array(axis) / np.linalg.norm(axis)
    target_pose.position.x = axis[0] * distance + current_pose.position.x
    target_pose.position.y = axis[1] * distance + current_pose.position.y
    target_pose.position.z = axis[2] * distance + current_pose.position.z
    target_pose.orientation = current_pose.orientation
    env.move_to_pose(target_pose)
    
move_in_direction(axis=joint_axis, distance=0.2)